In [2]:
import requests
import pandas as pd
from csv import writer
import json
import time
import glob
import os
from pathlib import Path
import tqdm
from twarc import Twarc
from nltk.tokenize import TweetTokenizer
import numpy as np
from random import sample
from collections import defaultdict,OrderedDict
from tqdm import tqdm
from os import listdir, makedirs, remove
from os.path import isfile, join

tk = TweetTokenizer()
tw = Twarc('asotxsa5cKuAeWjdYhZpABmN6', \
           'yhmMnNyE5bMx707yTcuS70NEoyFjvaPJnUCki1wHJG18XzLKYl', \
           '1116108383051046913-uzhQgRjVNAakb6pYKYUTCvW09donFA', \
           'XrHro3QvkuE0ivYXvQj9jufLDqk1EBNTcBnelXQ7hoM6P')
path = str(Path.cwd()) + '/data/'

In [4]:
fraction_we_need_to_sample = np.load("fraction_we_need_to_sample.npy", allow_pickle=True).tolist()

In [5]:
months = ['ukraine-russia-master/2022-02', 'ukraine-russia-master/2022-03', 'ukraine-russia-master/2022-04',
         'ukraine-russia-master/2022-05', 'ukraine-russia-master/2022-06', 'ukraine-russia-master/2022-07',
         'ukraine-russia-master/2022-08', 'ukraine-russia-master/2022-09', 'ukraine-russia-master/2022-10']

In [7]:
tweet_ids_per_day = defaultdict(list)
for month in tqdm(months):
    path = month
    month_num = month[22:]
    files = [f for f in listdir(path) if (isfile(join(path, f)))]
    for file in files:
        with open(join(path, file), 'r') as fp:
            for line in fp:
                date = file[20:-7]
                tweet_ids_per_day[date].append(line)
    tweet_ids_per_day_ordered = OrderedDict(sorted(tweet_ids_per_day.items()))
    
    tweet_ids_per_day_sampled = defaultdict(list)
    for day, tweet_ids in (tweet_ids_per_day_ordered.items()):
        tweet_ids_per_day_sampled[day] = sample(tweet_ids,fraction_we_need_to_sample[day])   
    
    np.save("sampled_tweet_ids/tweet_ids_per_day_sampled_"+str(month_num)+".npy", tweet_ids_per_day_sampled)
    del tweet_ids_per_day_ordered
    del tweet_ids_per_day
    del tweet_ids_per_day_sampled
    tweet_ids_per_day = defaultdict(list)

100%|█████████████████████████████████████████████| 9/9 [02:40<00:00, 17.83s/it]


In [25]:

files = [f for f in listdir(path) if (isfile(join(path, f)))]
files.sort()
for file in tqdm(files):
    tweet_ids_month = np.load(join(path, file), allow_pickle=True).tolist()
    for day, tweet_ids in tweet_ids_month.items():
        output = file[26:-4] + "-" + day[3:] + ".txt"
        with open(os.path.join(path, output), 'w') as f:
            for tweet_id in tweet_ids:
                f.write(f"{tweet_id}\n")

100%|█████████████████████████████████████████████| 9/9 [00:01<00:00,  4.96it/s]


In [44]:
import csv
keys_and_default = {'created_at': "NA",
    'id': "NA",
    'id_str': "NA",
    'full_text': "NA",
    'truncated': "NA",
    'display_text_range': "NA",
    'entities': "NA",
    'source': "NA",
    'in_reply_to_status_id': "NA",
    'in_reply_to_status_id_str': "NA",
    'in_reply_to_user_id': "NA",
    'in_reply_to_user_id_str': "NA",
    'in_reply_to_screen_name': "NA",
    'user': "NA",
    'geo': "NA",
    'coordinates': "NA",
    'place': "NA",
    'contributors': "NA",
    'is_quote_status': "NA",
    'quoted_status_id': "NA",
    'quoted_status_id_str': "NA",
    'quoted_status_permalink': "NA",
    'quoted_status': "NA",
    'retweet_count': "NA",
    'favorite_count': "NA",
    'favorited': "NA",
    'retweeted': "NA",
    'possibly_sensitive': "NA",
    'lang': "NA"}

def hydrate(path, file):
    with open(file, "w", newline="", encoding='utf-8') as outcsv:
    
        csvWriter = csv.DictWriter(outcsv, fieldnames=list(keys_and_default.keys()))
        csvWriter.writeheader()

        for tweet in tw.hydrate(open(path)):
            print(tweet)
            for key, default in keys_and_default.items():
                if key not in tweet:
                    tweet[key] = default

            csvWriter.writerow(tweet)
            break


In [45]:
path = 'sampled_tweet_ids'
hydrate(join(path, "2022-02-22.txt"), "sampled_tweets/2022-02-22.csv")

{'created_at': 'Tue Feb 22 04:50:05 +0000 2022', 'id': 1495984213631381510, 'id_str': '1495984213631381510', 'full_text': 'RT @KremlinRussia_E: "Russia has made and is making efforts to resolve all matters peacefully." The Security Council held a meeting at the…', 'truncated': False, 'display_text_range': [0, 139], 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'KremlinRussia_E', 'name': 'President of Russia', 'id': 205622130, 'id_str': '205622130', 'indices': [3, 19]}], 'urls': []}, 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1483152195306590210, 'id_str': '1483152195306590210', 'name': 'Ulisses Assis', 'screen_name': 'Ulisses_nobrega', 'location': '', 'description': '', 'url': None, 'entities': {'description': {'urls': []}}, 'p

ValueError: dict contains fields not in fieldnames: 'retweeted_status'

In [31]:
output.keys()

dict_keys(['created_at', 'id', 'id_str', 'full_text', 'truncated', 'display_text_range', 'entities', 'source', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place', 'contributors', 'retweeted_status', 'is_quote_status', 'retweet_count', 'favorite_count', 'favorited', 'retweeted', 'lang'])